In [35]:
bucket_name = 'iceberg-data-bucket'
catalog_name = 'iceberg_data_catalog'
schema_name = 'demodb_snappy_v1'
endpoint = "https://s3.jp-tok.cloud-object-storage.appdomain.cloud"
access_key = "9d05516331d44f38ba2c230ff036e8a0"
secret_key = "423c929fb85ff48809011f3264e10e7a87240150a5896b01"
raw_data_bucket_name = "raw-data-bucket"
cpd_user="cpadmin"
cpd_password="RkTyzx2Dgg6lw0HPfdwBjeiofq1sai8w"

In [36]:
conf=spark.sparkContext.getConf()
spark.stop()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_utc_timestamp
import base64,getpass
import pandas as pd
import io

wxd_username="cpadmin"
wxd_hms_username="ibmlhapikey_"+wxd_username
# wxd_hms_password="RkTyzx2Dgg6lw0HPfdwBjeiofq1sai8w"
wxd_hms_password="aW5zdGFuY2U6OnFrMjk2Mm9rdzVsOTc1d2Q="
string_to_encode=wxd_username+":"+wxd_hms_password
wxd_encoded_apikey="ZenApiKey "+base64.b64encode(string_to_encode.encode("utf-8")).decode("utf-8")

conf.setAll([("spark.hive.metastore.client.plain.username", wxd_hms_username), \
    ("spark.hive.metastore.client.plain.password", wxd_hms_password), \
    ('spark.hadoop.hive.wxd.user.name', wxd_username), \
    ("spark.hadoop.wxd.apikey", wxd_encoded_apikey), \
    ("spark.sql.parquet.compression.codec", "snappy"), \
    ("spark.sql.shuffle.partitions", "200"), \
    (f"spark.hadoop.fs.s3a.bucket.{raw_data_bucket_name}.endpoint", endpoint), \
    (f"spark.hadoop.fs.s3a.bucket.{raw_data_bucket_name}.access.key", access_key), \
    (f"spark.hadoop.fs.s3a.bucket.{raw_data_bucket_name}.secret.key", secret_key)
])

spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

In [37]:
spark

In [38]:
# for key, value in spark.sparkContext.getConf().getAll():
#     print(f"{key} = {value}")

In [39]:
spark.sql(f"show databases from {catalog_name}").show()

+--------------------+
|           namespace|
+--------------------+
|             default|
|                 kym|
|                test|
|              demodb|
|               stage|
|wxd_system_data_d...|
|wxd_system_data_d...|
|       demodb_snappy|
|    demodb_snappy_v1|
+--------------------+



In [8]:
spark.sql(f"create database if not exists {catalog_name}.{schema_name} LOCATION 's3a://{bucket_name}/{schema_name}/'")

DataFrame[]

In [26]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.dbgen_version (
    dv_version                STRING,
    dv_create_date            DATE,
    dv_create_time            STRING,
    dv_cmdline_args           STRING
)
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create customer_address table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.customer_address (
    ca_address_sk             integer,
    ca_address_id             varchar(16),
    ca_street_number          varchar(10),
    ca_street_name            varchar(60),
    ca_street_type            varchar(15),
    ca_suite_number           varchar(10),
    ca_city                   varchar(60),
    ca_county                 varchar(30),
    ca_state                  varchar(2),
    ca_zip                    varchar(10),
    ca_country                varchar(20),
    ca_gmt_offset             decimal(5,2),
    ca_location_type          varchar(20)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create customer_demographics table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.customer_demographics (
    cd_demo_sk                integer,
    cd_gender                 varchar(1),
    cd_marital_status         varchar(1),
    cd_education_status       varchar(20),
    cd_purchase_estimate      integer,
    cd_credit_rating          varchar(10),
    cd_dep_count              integer,
    cd_dep_employed_count     integer,
    cd_dep_college_count      integer
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create date_dim table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.date_dim (
    d_date_sk                 integer,
    d_date_id                 varchar(16),
    d_date                    date,
    d_month_seq               integer,
    d_week_seq                integer,
    d_quarter_seq             integer,
    d_year                    integer,
    d_dow                     integer,
    d_moy                     integer,
    d_dom                     integer,
    d_qoy                     integer,
    d_fy_year                 integer,
    d_fy_quarter_seq          integer,
    d_fy_week_seq             integer,
    d_day_name                varchar(9),
    d_quarter_name            varchar(6),
    d_holiday                 varchar(1),
    d_weekend                 varchar(1),
    d_following_holiday       varchar(1),
    d_first_dom               integer,
    d_last_dom                integer,
    d_same_day_ly             integer,
    d_same_day_lq             integer,
    d_current_day             varchar(1),
    d_current_week            varchar(1),
    d_current_month           varchar(1),
    d_current_quarter         varchar(1),
    d_current_year            varchar(1)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create warehouse table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.warehouse (
    w_warehouse_sk            integer,
    w_warehouse_id            varchar(16),
    w_warehouse_name          varchar(20),
    w_warehouse_sq_ft         integer,
    w_street_number           varchar(10),
    w_street_name             varchar(60),
    w_street_type             varchar(15),
    w_suite_number            varchar(10),
    w_city                    varchar(60),
    w_county                  varchar(30),
    w_state                   varchar(2),
    w_zip                     varchar(10),
    w_country                 varchar(20),
    w_gmt_offset              decimal(5,2)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create ship_mode table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.ship_mode (
    sm_ship_mode_sk           integer,
    sm_ship_mode_id           varchar(16),
    sm_type                   varchar(30),
    sm_code                   varchar(10),
    sm_carrier                varchar(20),
    sm_contract               varchar(20)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create time_dim table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.time_dim (
    t_time_sk                 integer,
    t_time_id                 varchar(16),
    t_time                    integer,
    t_hour                    integer,
    t_minute                  integer,
    t_second                  integer,
    t_am_pm                   varchar(2),
    t_shift                   varchar(20),
    t_sub_shift               varchar(20),
    t_meal_time               varchar(20)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create reason table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.reason (
    r_reason_sk               integer,
    r_reason_id               varchar(16),
    r_reason_desc             varchar(100)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create income_band table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.income_band (
    ib_income_band_sk         integer,
    ib_lower_bound            integer,
    ib_upper_bound            integer
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create item table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.item (
    i_item_sk                 integer,
    i_item_id                 varchar(16),
    i_rec_start_date          date,
    i_rec_end_date            date,
    i_item_desc               varchar(200),
    i_current_price           decimal(7,2),
    i_wholesale_cost          decimal(7,2),
    i_brand_id                integer,
    i_brand                   varchar(50),
    i_class_id                integer,
    i_class                   varchar(50),
    i_category_id             integer,
    i_category                varchar(50),
    i_manufact_id             integer,
    i_manufact                varchar(50),
    i_size                    varchar(20),
    i_formulation             varchar(20),
    i_color                   varchar(20),
    i_units                   varchar(10),
    i_container               varchar(10),
    i_manager_id              integer,
    i_product_name            varchar(50)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create store table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.store (
    s_store_sk                integer,
    s_store_id                varchar(16),
    s_rec_start_date          date,
    s_rec_end_date            date,
    s_closed_date_sk          integer,
    s_store_name              varchar(50),
    s_number_employees        integer,
    s_floor_space             integer,
    s_hours                   varchar(20),
    s_manager                 varchar(40),
    s_market_id               integer,
    s_geography_class         varchar(100),
    s_market_desc             varchar(100),
    s_market_manager          varchar(40),
    s_division_id             integer,
    s_division_name           varchar(50),
    s_company_id              integer,
    s_company_name            varchar(50),
    s_street_number           varchar(10),
    s_street_name             varchar(60),
    s_street_type             varchar(15),
    s_suite_number            varchar(10),
    s_city                    varchar(60),
    s_county                  varchar(30),
    s_state                   varchar(2),
    s_zip                     varchar(10),
    s_country                 varchar(20),
    s_gmt_offset              decimal(5,2),
    s_tax_percentage          decimal(5,2)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create call_center table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.call_center (
    cc_call_center_sk         integer,
    cc_call_center_id         varchar(16),
    cc_rec_start_date         date,
    cc_rec_end_date           date,
    cc_closed_date_sk         integer,
    cc_open_date_sk           integer,
    cc_name                   varchar(50),
    cc_class                  varchar(50),
    cc_employees              integer,
    cc_sq_ft                  integer,
    cc_hours                  varchar(20),
    cc_manager                varchar(40),
    cc_mkt_id                 integer,
    cc_mkt_class              varchar(50),
    cc_mkt_desc               varchar(100),
    cc_market_manager         varchar(40),
    cc_division               integer,
    cc_division_name          varchar(50),
    cc_company                integer,
    cc_company_name           varchar(50),
    cc_street_number          varchar(10),
    cc_street_name            varchar(60),
    cc_street_type            varchar(15),
    cc_suite_number           varchar(10),
    cc_city                   varchar(60),
    cc_county                 varchar(30),
    cc_state                  varchar(2),
    cc_zip                    varchar(10),
    cc_country                varchar(20),
    cc_gmt_offset             decimal(5,2),
    cc_tax_percentage         decimal(5,2)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create customer table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.customer (
    c_customer_sk             integer,
    c_customer_id             varchar(16),
    c_current_cdemo_sk        integer,
    c_current_hdemo_sk        integer,
    c_current_addr_sk         integer,
    c_first_shipto_date_sk    integer,
    c_first_sales_date_sk     integer,
    c_salutation              varchar(10),
    c_first_name              varchar(20),
    c_last_name               varchar(30),
    c_preferred_cust_flag     varchar(1),
    c_birth_day               integer,
    c_birth_month             integer,
    c_birth_year              integer,
    c_birth_country           varchar(20),
    c_login                   varchar(13),
    c_email_address           varchar(50),
    c_last_review_date        varchar(10)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create web_site table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.web_site (
    web_site_sk               integer,
    web_site_id               varchar(16),
    web_rec_start_date        date,
    web_rec_end_date          date,
    web_name                  varchar(50),
    web_open_date_sk          integer,
    web_close_date_sk         integer,
    web_class                 varchar(50),
    web_manager               varchar(40),
    web_mkt_id                integer,
    web_mkt_class             varchar(50),
    web_mkt_desc              varchar(100),
    web_market_manager        varchar(40),
    web_company_id            integer,
    web_company_name          varchar(50),
    web_street_number         varchar(10),
    web_street_name           varchar(60),
    web_street_type           varchar(15),
    web_suite_number          varchar(10),
    web_city                  varchar(60),
    web_county                varchar(30),
    web_state                 varchar(2),
    web_zip                   varchar(10),
    web_country               varchar(20),
    web_gmt_offset            decimal(5,2),
    web_tax_percentage        decimal(5,2)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create store_returns table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.store_returns (
    sr_returned_date_sk       integer,
    sr_return_time_sk         integer,
    sr_item_sk                integer,
    sr_customer_sk            integer,
    sr_cdemo_sk               integer,
    sr_hdemo_sk               integer,
    sr_addr_sk                integer,
    sr_store_sk               integer,
    sr_reason_sk              integer,
    sr_ticket_number          integer,
    sr_return_quantity        integer,
    sr_return_amt             decimal(7,2),
    sr_return_tax             decimal(7,2),
    sr_return_amt_inc_tax     decimal(7,2),
    sr_fee                    decimal(7,2),
    sr_return_ship_cost       decimal(7,2),
    sr_refunded_cash          decimal(7,2),
    sr_reversed_charge        decimal(7,2),
    sr_store_credit           decimal(7,2),
    sr_net_loss               decimal(7,2)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create household_demographics table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.household_demographics (
    hd_demo_sk                integer,
    hd_income_band_sk         integer,
    hd_buy_potential          varchar(15),
    hd_dep_count              integer,
    hd_vehicle_count          integer
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create web_page table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.web_page (
    wp_web_page_sk            integer,
    wp_web_page_id            varchar(16),
    wp_rec_start_date         date,
    wp_rec_end_date           date,
    wp_creation_date_sk       integer,
    wp_access_date_sk         integer,
    wp_autogen_flag           varchar(1),
    wp_customer_sk            integer,
    wp_url                    varchar(100),
    wp_type                   varchar(50),
    wp_char_count             integer,
    wp_link_count             integer,
    wp_image_count            integer,
    wp_max_ad_count           integer
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create promotion table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.promotion (
    p_promo_sk                integer,
    p_promo_id                varchar(16),
    p_start_date_sk           integer,
    p_end_date_sk             integer,
    p_item_sk                 integer,
    p_cost                    decimal(15,2),
    p_response_target         integer,
    p_promo_name              varchar(50),
    p_channel_dmail           varchar(1),
    p_channel_email           varchar(1),
    p_channel_catalog         varchar(1),
    p_channel_tv              varchar(1),
    p_channel_radio           varchar(1),
    p_channel_press           varchar(1),
    p_channel_event           varchar(1),
    p_channel_demo            varchar(1),
    p_channel_details         varchar(100),
    p_purpose                 varchar(15),
    p_discount_active         varchar(1)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create catalog_page table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.catalog_page (
    cp_catalog_page_sk        integer,
    cp_catalog_page_id        varchar(16),
    cp_start_date_sk          integer,
    cp_end_date_sk            integer,
    cp_department             varchar(50),
    cp_catalog_number         integer,
    cp_catalog_page_number    integer,
    cp_description            varchar(100),
    cp_type                   varchar(100)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create inventory table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.inventory (
    inv_date_sk               integer,
    inv_item_sk               integer,
    inv_warehouse_sk          integer,
    inv_quantity_on_hand      integer
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create catalog_returns table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.catalog_returns (
    cr_returned_date_sk       integer,
    cr_returned_time_sk       integer,
    cr_item_sk                integer,
    cr_refunded_customer_sk   integer,
    cr_refunded_cdemo_sk      integer,
    cr_refunded_hdemo_sk      integer,
    cr_refunded_addr_sk       integer,
    cr_returning_customer_sk  integer,
    cr_returning_cdemo_sk     integer,
    cr_returning_hdemo_sk     integer,
    cr_returning_addr_sk      integer,
    cr_call_center_sk         integer,
    cr_catalog_page_sk        integer,
    cr_ship_mode_sk           integer,
    cr_warehouse_sk           integer,
    cr_reason_sk              integer,
    cr_order_number           integer,
    cr_return_quantity        integer,
    cr_return_amount          decimal(7,2),
    cr_return_tax             decimal(7,2),
    cr_return_amt_inc_tax     decimal(7,2),
    cr_fee                    decimal(7,2),
    cr_return_ship_cost       decimal(7,2),
    cr_refunded_cash          decimal(7,2),
    cr_reversed_charge        decimal(7,2),
    cr_store_credit           decimal(7,2),
    cr_net_loss               decimal(7,2)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create web_returns table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.web_returns (
    wr_returned_date_sk       integer,
    wr_returned_time_sk       integer,
    wr_item_sk                integer,
    wr_refunded_customer_sk   integer,
    wr_refunded_cdemo_sk      integer,
    wr_refunded_hdemo_sk      integer,
    wr_refunded_addr_sk       integer,
    wr_returning_customer_sk  integer,
    wr_returning_cdemo_sk     integer,
    wr_returning_hdemo_sk     integer,
    wr_returning_addr_sk      integer,
    wr_web_page_sk            integer,
    wr_reason_sk              integer,
    wr_order_number           integer,
    wr_return_quantity        integer,
    wr_return_amt             decimal(7,2),
    wr_return_tax             decimal(7,2),
    wr_return_amt_inc_tax     decimal(7,2),
    wr_fee                    decimal(7,2),
    wr_return_ship_cost       decimal(7,2),
    wr_refunded_cash          decimal(7,2),
    wr_reversed_charge        decimal(7,2),
    wr_account_credit         decimal(7,2),
    wr_net_loss               decimal(7,2)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create web_sales table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.web_sales (
    ws_sold_date_sk           integer,
    ws_sold_time_sk           integer,
    ws_ship_date_sk           integer,
    ws_item_sk                integer,
    ws_bill_customer_sk       integer,
    ws_bill_cdemo_sk          integer,
    ws_bill_hdemo_sk          integer,
    ws_bill_addr_sk           integer,
    ws_ship_customer_sk       integer,
    ws_ship_cdemo_sk          integer,
    ws_ship_hdemo_sk          integer,
    ws_ship_addr_sk           integer,
    ws_web_page_sk            integer,
    ws_web_site_sk            integer,
    ws_ship_mode_sk           integer,
    ws_warehouse_sk           integer,
    ws_promo_sk               integer,
    ws_order_number           integer,
    ws_quantity               integer,
    ws_wholesale_cost         decimal(7,2),
    ws_list_price             decimal(7,2),
    ws_sales_price            decimal(7,2),
    ws_ext_discount_amt       decimal(7,2),
    ws_ext_sales_price        decimal(7,2),
    ws_ext_wholesale_cost     decimal(7,2),
    ws_ext_list_price         decimal(7,2),
    ws_ext_tax                decimal(7,2),
    ws_coupon_amt             decimal(7,2),
    ws_ext_ship_cost          decimal(7,2),
    ws_net_paid               decimal(7,2),
    ws_net_paid_inc_tax       decimal(7,2),
    ws_net_paid_inc_ship      decimal(7,2),
    ws_net_paid_inc_ship_tax  decimal(7,2),
    ws_net_profit             decimal(7,2)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create catalog_sales table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.catalog_sales (
    cs_sold_date_sk           integer,
    cs_sold_time_sk           integer,
    cs_ship_date_sk           integer,
    cs_bill_customer_sk       integer,
    cs_bill_cdemo_sk          integer,
    cs_bill_hdemo_sk          integer,
    cs_bill_addr_sk           integer,
    cs_ship_customer_sk       integer,
    cs_ship_cdemo_sk          integer,
    cs_ship_hdemo_sk          integer,
    cs_ship_addr_sk           integer,
    cs_call_center_sk         integer,
    cs_catalog_page_sk        integer,
    cs_ship_mode_sk           integer,
    cs_warehouse_sk           integer,
    cs_item_sk                integer,
    cs_promo_sk               integer,
    cs_order_number           integer,
    cs_quantity               integer,
    cs_wholesale_cost         decimal(7,2),
    cs_list_price             decimal(7,2),
    cs_sales_price            decimal(7,2),
    cs_ext_discount_amt       decimal(7,2),
    cs_ext_sales_price        decimal(7,2),
    cs_ext_wholesale_cost     decimal(7,2),
    cs_ext_list_price         decimal(7,2),
    cs_ext_tax                decimal(7,2),
    cs_coupon_amt             decimal(7,2),
    cs_ext_ship_cost          decimal(7,2),
    cs_net_paid               decimal(7,2),
    cs_net_paid_inc_tax       decimal(7,2),
    cs_net_paid_inc_ship      decimal(7,2),
    cs_net_paid_inc_ship_tax  decimal(7,2),
    cs_net_profit             decimal(7,2)
) 
USING PARQUET
OPTIONS ('compression'='snappy');""")

spark.sql(f"""

-- Create store_sales table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.store_sales (
    ss_sold_date_sk           integer,
    ss_sold_time_sk           integer,
    ss_item_sk                integer,
    ss_customer_sk            integer,
    ss_cdemo_sk               integer,
    ss_hdemo_sk               integer,
    ss_addr_sk                integer,
    ss_store_sk               integer,
    ss_promo_sk               integer,
    ss_ticket_number          integer,
    ss_quantity               integer,
    ss_wholesale_cost         decimal(7,2),
    ss_list_price             decimal(7,2),
    ss_sales_price            decimal(7,2),
    ss_ext_discount_amt       decimal(7,2),
    ss_ext_sales_price        decimal(7,2),
    ss_ext_wholesale_cost     decimal(7,2),
    ss_ext_list_price         decimal(7,2),
    ss_ext_tax                decimal(7,2),
    ss_coupon_amt             decimal(7,2),
    ss_net_paid               decimal(7,2),
    ss_net_paid_inc_tax       decimal(7,2),
    ss_net_profit             decimal(7,2)
) 
USING PARQUET
OPTIONS ('compression'='snappy');

""")


DataFrame[]

In [9]:
spark.sql(f"""create table if not exists {catalog_name}.{schema_name}.testTable(id INTEGER, name VARCHAR(10), age INTEGER, salary DECIMAL(10, 2)) using iceberg TBLPROPERTIES (
    'write.parquet.compression-codec'='snappy'
)""").show()
spark.sql(f"insert into {catalog_name}.{schema_name}.testTable values(1,'Alan',23,3400.00),(2,'Ben',30,5500.00),(3,'Chen',35,6500.00)")
spark.sql(f"select * from {catalog_name}.{schema_name}.testTable").show()

++
||
++
++

+---+----+---+-------+
| id|name|age| salary|
+---+----+---+-------+
|  1|Alan| 23|3400.00|
|  2| Ben| 30|5500.00|
|  3|Chen| 35|6500.00|
|  1|Alan| 23|3400.00|
|  2| Ben| 30|5500.00|
|  3|Chen| 35|6500.00|
+---+----+---+-------+



In [8]:
spark.sql(f"SELECT * FROM {catalog_name}.{schema_name}.testTable.files")

DataFrame[content: int, file_path: string, file_format: string, spec_id: int, record_count: bigint, file_size_in_bytes: bigint, column_sizes: map<int,bigint>, value_counts: map<int,bigint>, null_value_counts: map<int,bigint>, nan_value_counts: map<int,bigint>, lower_bounds: map<int,binary>, upper_bounds: map<int,binary>, key_metadata: binary, split_offsets: array<bigint>, equality_ids: array<int>, sort_order_id: int, readable_metrics: struct<age:struct<column_size:bigint,value_count:bigint,null_value_count:bigint,nan_value_count:bigint,lower_bound:int,upper_bound:int>,id:struct<column_size:bigint,value_count:bigint,null_value_count:bigint,nan_value_count:bigint,lower_bound:int,upper_bound:int>,name:struct<column_size:bigint,value_count:bigint,null_value_count:bigint,nan_value_count:bigint,lower_bound:string,upper_bound:string>,salary:struct<column_size:bigint,value_count:bigint,null_value_count:bigint,nan_value_count:bigint,lower_bound:decimal(10,2),upper_bound:decimal(10,2)>>]

In [25]:
file_path = "s3a://iceberg-data-bucket/demodb_snappy_v1/testTable/data/00001-6-3e3d527e-3fb9-45fc-b2ad-590d7019e1cb-0-00001.parquet"

# 파일 읽기
df = spark.read.format("parquet").load(file_path)

# Parquet 파일의 메타데이터 출력
print(df._jdf.queryExecution().toString())


== Parsed Logical Plan ==
Relation [id#150,name#151,age#152,salary#153] parquet

== Analyzed Logical Plan ==
id: int, name: string, age: int, salary: decimal(10,2)
Relation [id#150,name#151,age#152,salary#153] parquet

== Optimized Logical Plan ==
Relation [id#150,name#151,age#152,salary#153] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [id#150,name#151,age#152,salary#153] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[s3a://iceberg-data-bucket/demodb_snappy_v1/testTable/data/00001-6-3e3d..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:int,name:string,age:int,salary:decimal(10,2)>



In [12]:
file_path = "s3a://iceberg-data-bucket/demodb_snappy/testTable/data/00000-0-47a252bc-a2e1-45b2-80ef-133f4d422a44-0-00001.parquet"

# 파일 읽기
df = spark.read.format("parquet").load(file_path)

# Parquet 파일의 메타데이터 출력
print(df._jdf.queryExecution().toString())


== Parsed Logical Plan ==
Relation [id#101,name#102,age#103,salary#104] parquet

== Analyzed Logical Plan ==
id: int, name: string, age: int, salary: decimal(10,2)
Relation [id#101,name#102,age#103,salary#104] parquet

== Optimized Logical Plan ==
Relation [id#101,name#102,age#103,salary#104] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [id#101,name#102,age#103,salary#104] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[s3a://iceberg-data-bucket/demodb_snappy/testTable/data/00000-0-47a252b..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:int,name:string,age:int,salary:decimal(10,2)>



In [6]:
file_path = "s3a://iceberg-data-bucket/demodb_snappy/catalog_page/data/00000-34-fc9760c3-dab3-4602-aa7e-5ff61d69fea4-0-00001.parquet"

# 파일 읽기
df = spark.read.format("parquet").load(file_path)

# Parquet 파일의 메타데이터 출력
print(df._jdf.queryExecution().toString())


== Parsed Logical Plan ==
Relation [cp_catalog_page_sk#11,cp_catalog_page_id#12,cp_start_date_sk#13,cp_end_date_sk#14,cp_department#15,cp_catalog_number#16,cp_catalog_page_number#17,cp_description#18,cp_type#19] parquet

== Analyzed Logical Plan ==
cp_catalog_page_sk: int, cp_catalog_page_id: string, cp_start_date_sk: int, cp_end_date_sk: int, cp_department: string, cp_catalog_number: int, cp_catalog_page_number: int, cp_description: string, cp_type: string
Relation [cp_catalog_page_sk#11,cp_catalog_page_id#12,cp_start_date_sk#13,cp_end_date_sk#14,cp_department#15,cp_catalog_number#16,cp_catalog_page_number#17,cp_description#18,cp_type#19] parquet

== Optimized Logical Plan ==
Relation [cp_catalog_page_sk#11,cp_catalog_page_id#12,cp_start_date_sk#13,cp_end_date_sk#14,cp_department#15,cp_catalog_number#16,cp_catalog_page_number#17,cp_description#18,cp_type#19] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [cp_catalog_page_sk#11,cp_catalog_page_id#12,cp_start_date_s

In [40]:
import boto3
import time

# Create S3 client (for IBM COS)
s3 = boto3.client(
    's3',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    endpoint_url=endpoint
)

def human_readable_size(size_in_bytes):
    """
    Convert size in bytes to a human-readable format (e.g., KB, MB, GB).
    """
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_in_bytes < 1024:
            return f"{size_in_bytes:.2f} {unit}"
        size_in_bytes /= 1024
    return f"{size_in_bytes:.2f} TB"

def list_dat_files(bucket_name):
    """S3 버킷에서 .dat 파일 목록을 반환합니다."""
    try:
        response = s3.list_objects_v2(Bucket=bucket_name)
        files = response.get('Contents', [])
        dat_files = [{'Key': file['Key'], 'Size': file['Size']} for file in files if file['Key'].endswith('.dat')]
        return dat_files
    except Exception as e:
        print(f"파일 목록을 가져오는 중 오류 발생: {e}")
        return []

def drop_table_if_exists(catalog_name, schema_name, table_name):
    """Iceberg 테이블이 존재하면 삭제합니다."""
    try:
        # 테이블 존재 여부 확인
        table_exists = spark.catalog.tableExists(f"{catalog_name}.{schema_name}.{table_name}")
        if table_exists:
            # 테이블 삭제
            spark.sql(f"DROP TABLE IF EXISTS {catalog_name}.{schema_name}.{table_name}")
            print(f"{table_name} 테이블 삭제 완료")
        else:
            print(f"{table_name} 테이블이 존재하지 않습니다.")
    except Exception as e:
        print(f"테이블 삭제 중 오류 발생: {e}")

def get_iceberg_schema(table_name):
    """Iceberg 테이블의 스키마를 반환합니다."""
    try:
        # Iceberg 테이블의 스키마 가져오기
        schema = spark.sql(f"DESCRIBE TABLE {table_name}").select("col_name").rdd.flatMap(lambda x: x).collect()
        # 컬럼 이름만 반환 (메타정보 제외)
        return [col for col in schema if not col.startswith("#")]
    except Exception as e:
        print(f"❌ {table_name} 테이블의 스키마를 가져오는 중 오류 발생: {e}")
        return None

def cast_dataframe_to_schema(df, schema_columns, schema_dtypes):
    """DataFrame을 Iceberg 스키마에 맞게 캐스팅합니다."""
    from pyspark.sql.functions import col

    casted_columns = []
    for col_name, col_type in zip(schema_columns, schema_dtypes):
        if col_name in df.columns:
            # 현재 컬럼을 Iceberg 스키마에 맞는 데이터 타입으로 캐스팅
            casted_columns.append(col(col_name).cast(col_type).alias(col_name))
        else:
            # 누락된 컬럼은 null로 추가
            casted_columns.append(lit(None).cast(col_type).alias(col_name))

    # 새로운 DataFrame 생성
    return df.select(*casted_columns)

def ingest_data_to_iceberg(dat_files, catalog_name, schema_name):
    """주어진 .dat 파일들을 읽어 Iceberg 테이블에 데이터를 적재합니다."""
    total_size = 0  # 총 파일 크기 초기화
    print("="*50)
    for file_info in dat_files:
        file = file_info['Key']
        file_size = file_info['Size']
        if file in ['dbgen_version']: continue
        total_size += file_size  # 파일 크기 합산
        
        # 파일 이름에서 테이블 이름 추출
        iceberg_table = file.split('.dat')[0]
        full_table_name = f"{catalog_name}.{schema_name}.{iceberg_table}"

        # 현재 파일 작업 시작을 표시
        print(f"현재 작업 중: {file} -> 테이블명: {iceberg_table} (파일 크기: {human_readable_size(file_size)})")
        print("="*50)

        # 작업 시작 시간 기록
        start_time = time.time()

        # Iceberg 테이블 스키마 가져오기
        schema_df = spark.sql(f"DESCRIBE TABLE {full_table_name}")
        schema_columns = schema_df.select("col_name").rdd.flatMap(lambda x: x).collect()
        schema_dtypes = schema_df.select("data_type").rdd.flatMap(lambda x: x).collect()
        
        # 테이블 삭제 (존재하면)
        # drop_table_if_exists(catalog_name, schema_name, iceberg_table)

        # .dat 파일 읽기
        df = spark.read \
            .format('csv') \
            .option('header', False) \
            .option('inferSchema', True) \
            .option('delimiter', '|') \
            .csv(f"s3a://{raw_data_bucket_name}/{file}")
        # df.printSchema()
        # df.show(3)
        # df = df.toDF(*schema_columns)

        # 컬럼 이름 적용 (Iceberg 테이블의 스키마에 맞게 설정)
        if len(df.columns) > len(schema_columns):
            df = df.select(df.columns[:len(schema_columns)])
        df = df.toDF(*schema_columns)

        # DataFrame을 Iceberg 테이블의 데이터 타입으로 캐스팅
        try:
            df = cast_dataframe_to_schema(df, schema_columns, schema_dtypes)
        except Exception as e:
            print(f"❌ 데이터 타입 변환 중 오류 발생: {e}")
            continue

        # Iceberg 테이블에 데이터 적재
        try:
            # df.writeTo(full_table_name) \
            # .tableProperty("write.format.default", "parquet") \
            # .tableProperty("write.parquet.compression-codec", "snappy") \
            # .append()

            df.writeTo(full_table_name) \
            .option("compression", "snappy") \
            .append()
            # df.writeTo(f"{catalog_name}.{schema_name}.{iceberg_table}") \
            #     .tableProperty("write.format.default", "parquet") \
            #     .createOrReplace()
            print(f"✅ {iceberg_table} 테이블에 데이터 적재 완료")

        except Exception as e:
            print(f"❌ {file}의 데이터를 적재하는 중 오류 발생: {e}")
            continue
        
        # 작업 종료 시간 및 경과 시간 계산
        elapsed_time = time.time() - start_time
        print(f"작업 완료 시간: {elapsed_time:.2f} 초")
        print("="*50)
        
    # 총 파일 크기 출력
    print(f"\n📊 전체 .dat 파일의 크기 합계: {human_readable_size(total_size)}")

In [41]:
# .dat 파일 목록 가져오기
dat_files = list_dat_files(raw_data_bucket_name)
dat_files

[{'Key': 'call_center.dat', 'Size': 9356},
 {'Key': 'catalog_page.dat', 'Size': 2857922},
 {'Key': 'catalog_returns.dat', 'Size': 2279225314},
 {'Key': 'catalog_sales.dat', 'Size': 31016462258},
 {'Key': 'customer.dat', 'Size': 269515941},
 {'Key': 'customer_address.dat', 'Size': 111154196},
 {'Key': 'customer_demographics.dat', 'Size': 80660096},
 {'Key': 'date_dim.dat', 'Size': 10317438},
 {'Key': 'dbgen_version.dat', 'Size': 88},
 {'Key': 'household_demographics.dat', 'Size': 151653},
 {'Key': 'income_band.dat', 'Size': 328},
 {'Key': 'inventory.dat', 'Size': 8626269134},
 {'Key': 'item.dat', 'Size': 58366791},
 {'Key': 'promotion.dat', 'Size': 124973},
 {'Key': 'reason.dat', 'Size': 1959},
 {'Key': 'ship_mode.dat', 'Size': 1113},
 {'Key': 'store.dat', 'Size': 106820},
 {'Key': 'store_returns.dat', 'Size': 3483867155},
 {'Key': 'store_sales.dat', 'Size': 40959624908},
 {'Key': 'time_dim.dat', 'Size': 5107780},
 {'Key': 'warehouse.dat', 'Size': 1782},
 {'Key': 'web_page.dat', 'Size':

In [12]:
spark.sql(f"DROP DATABASE IF EXISTS {catalog_name}.{schema_name}")

DataFrame[]

In [22]:
drop_table_if_exists(catalog_name, schema_name, 'testtable')

testtable 테이블 삭제 완료


In [9]:
dat_files = list_dat_files(raw_data_bucket_name)
for dat_file in dat_files:
    tbl_name = dat_file['Key'].split('.dat')[0]
    drop_table_if_exists(catalog_name, schema_name, tbl_name)
    # print(tbl_name)

call_center 테이블 삭제 완료
catalog_page 테이블 삭제 완료
catalog_returns 테이블 삭제 완료
catalog_sales 테이블 삭제 완료
customer 테이블 삭제 완료
customer_address 테이블 삭제 완료
customer_demographics 테이블 삭제 완료
date_dim 테이블 삭제 완료
dbgen_version 테이블 삭제 완료
household_demographics 테이블 삭제 완료
income_band 테이블 삭제 완료
inventory 테이블 삭제 완료
item 테이블 삭제 완료
promotion 테이블 삭제 완료
reason 테이블 삭제 완료
ship_mode 테이블 삭제 완료
store 테이블 삭제 완료
store_returns 테이블 삭제 완료
store_sales 테이블 삭제 완료
time_dim 테이블 삭제 완료
warehouse 테이블 삭제 완료
web_page 테이블 삭제 완료
web_returns 테이블 삭제 완료
web_sales 테이블 삭제 완료
web_site 테이블 삭제 완료


In [ ]:
# .dat 파일 목록 가져오기
dat_files = list_dat_files(raw_data_bucket_name)

# Iceberg 테이블에 데이터 적재
ingest_data_to_iceberg(dat_files, catalog_name, schema_name)

현재 작업 중: call_center.dat -> 테이블명: call_center (파일 크기: 9.14 KB)
✅ call_center 테이블에 데이터 적재 완료
작업 완료 시간: 20.65 초
현재 작업 중: catalog_page.dat -> 테이블명: catalog_page (파일 크기: 2.73 MB)
✅ catalog_page 테이블에 데이터 적재 완료
작업 완료 시간: 14.14 초
현재 작업 중: catalog_returns.dat -> 테이블명: catalog_returns (파일 크기: 2.12 GB)
✅ catalog_returns 테이블에 데이터 적재 완료
작업 완료 시간: 226.23 초
현재 작업 중: catalog_sales.dat -> 테이블명: catalog_sales (파일 크기: 28.89 GB)
✅ catalog_sales 테이블에 데이터 적재 완료
작업 완료 시간: 2256.28 초
현재 작업 중: customer.dat -> 테이블명: customer (파일 크기: 257.03 MB)
✅ customer 테이블에 데이터 적재 완료
작업 완료 시간: 30.38 초
현재 작업 중: customer_address.dat -> 테이블명: customer_address (파일 크기: 106.00 MB)
✅ customer_address 테이블에 데이터 적재 완료
작업 완료 시간: 23.11 초
현재 작업 중: customer_demographics.dat -> 테이블명: customer_demographics (파일 크기: 76.92 MB)
✅ customer_demographics 테이블에 데이터 적재 완료
작업 완료 시간: 16.79 초
현재 작업 중: date_dim.dat -> 테이블명: date_dim (파일 크기: 9.84 MB)
✅ date_dim 테이블에 데이터 적재 완료
작업 완료 시간: 12.37 초
현재 작업 중: dbgen_version.dat -> 테이블명: dbgen_version (파일 크기: 88.00

In [43]:
spark.sql(f"""

-- Create date_dim table
CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.date_dim_new (
    d_date_sk                 integer,
    d_date_id                 varchar(16),
    d_date                    date,
    d_month_seq               integer,
    d_week_seq                integer,
    d_quarter_seq             integer,
    d_year                    integer,
    d_dow                     integer,
    d_moy                     integer,
    d_dom                     integer,
    d_qoy                     integer,
    d_fy_year                 integer,
    d_fy_quarter_seq          integer,
    d_fy_week_seq             integer,
    d_day_name                varchar(9),
    d_quarter_name            varchar(6),
    d_holiday                 varchar(1),
    d_weekend                 varchar(1),
    d_following_holiday       varchar(1),
    d_first_dom               integer,
    d_last_dom                integer,
    d_same_day_ly             integer,
    d_same_day_lq             integer,
    d_current_day             varchar(1),
    d_current_week            varchar(1),
    d_current_month           varchar(1),
    d_current_quarter         varchar(1),
    d_current_year            varchar(1)
) 
USING PARQUET
PARTITIONED BY (d_year, d_moy)
OPTIONS ('compression'='snappy');""")

DataFrame[]

In [21]:
dat_files = list_dat_files(raw_data_bucket_name)
dat_files[7:8]

[{'Key': 'date_dim.dat', 'Size': 10317438}]

In [42]:
drop_table_if_exists(catalog_name, schema_name, 'date_dim_new')

date_dim_new 테이블 삭제 완료


In [25]:
ingest_data_to_iceberg(dat_files[7:8], catalog_name, schema_name)

현재 작업 중: date_dim.dat -> 테이블명: date_dim (파일 크기: 9.84 MB)
✅ date_dim 테이블에 데이터 적재 완료
작업 완료 시간: 16.87 초

📊 전체 .dat 파일의 크기 합계: 9.84 MB


In [29]:
spark.sql(f"""
INSERT INTO {catalog_name}.{schema_name}.date_dim_new
SELECT * FROM {catalog_name}.{schema_name}.date_dim;
""")


DataFrame[]

In [30]:
import time
start=time.time()
elapsed_time = time.time() - start

# 경과 시간 출력
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 0.00 seconds


In [ ]:
import time

df = spark.table(f"{catalog_name}.{schema_name}.date_dim")

start=time.time()
# 데이터를 새로운 파티션에 쓰기
df.repartition("d_year", "d_moy") \
  .write \
  .format("parquet") \
  .mode("append") \
  .partitionBy("d_year", "d_moy") \
  .saveAsTable(f"{catalog_name}.{schema_name}.date_dim_new")
# 경과 시간 출력
elapsed_time = time.time() - start
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 388.64 seconds


In [33]:
import time

df = spark.table(f"{catalog_name}.{schema_name}.date_dim")

start=time.time()
# 데이터를 새로운 파티션에 쓰기
df.repartition("d_year", "d_moy") \
  .write \
  .format("parquet") \
  .mode("append") \
  .partitionBy("d_year", "d_moy") \
  .saveAsTable(f"{catalog_name}.{schema_name}.date_dim_new")
# 경과 시간 출력
elapsed_time = time.time() - start
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 493.15 seconds


In [34]:
for key, value in spark.sparkContext.getConf().getAll():
    print(f"{key} = {value}")

spark.hive.metastore.truststore.password = changeit
spark.hive.metastore.truststore.type = JKS
spark.eventLog.enabled = true
spark.network.crypto.keyLength = 256
spark.driver.memory = 3600M
spark.network.crypto.enabled = true
spark.driver.extraClassPath = /home/spark/user_home/dbdrivers/*:/cc-home/_global_/dbdrivers/*:/cc-home/_global_/dbdrivers/jdbc/default/*:/home/spark/shared/user-libs/spark/*:/home/spark/shared/user-libs/common/*:/home/spark/shared/user-libs/connectors/*:/home/spark/space/assets/data_asset/*:/project_data/data_asset/*:/opt/ibm/spark/external-jars/iceberg-spark-runtime-iceberg-spark.jar:/opt/ibm/connectors/parquet-encryption/*:/opt/ibm/third-party/libs/spark2/*:/opt/ibm/third-party/libs/common/*:/opt/ibm/third-party/libs/connectors/*:/opt/ibm/connectors/others-db-drivers/*:/opt/ibm/connectors/wdp-connector-driver/*:/opt/ibm/connectors/db2/*:/opt/ibm/connectors/data-engine/*:/opt/ibm/connectors/wxd/*:/opt/ibm/connectors/snowflake/*:/opt/ibm/spark/external-jars/*
spar